# Get the NVD feed into the MongoDB

In [1]:
from pprint import pprint

from pymongo import MongoClient

from nvd_connector import NVDFeed, NVDEntry

In [2]:
mongo_db_name = "nvd"

In [3]:
mongo_client = MongoClient("127.0.0.1")
db = mongo_client['admin']
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)
dblist = mongo_client.list_database_names()
print(f'Existing Databases: {dblist}')
if mongo_db_name in dblist:
    print("The database exists.")

mongo_db = mongo_client[mongo_db_name]

{'asserts': {'msg': 0,
             'regular': 0,
             'rollovers': 0,
             'tripwire': 0,
             'user': 63,
             'warning': 0},
 'catalogStats': {'capped': 0,
                  'collections': 7,
                  'internalCollections': 4,
                  'internalViews': 0,
                  'timeseries': 0,
                  'views': 14},
 'connections': {'active': 2,
                 'available': 51197,
                 'awaitingTopologyChanges': 1,
                 'current': 3,
                 'exhaustHello': 0,
                 'exhaustIsMaster': 0,
                 'threaded': 3,
                 'totalCreated': 21},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCatchUpsA

In [4]:
years = [2022]
feed = NVDFeed(*years, directory='nvd_files')

In [5]:
def nvd_entry_to_dict(nvd_entry: NVDEntry) -> dict:
    data = vars(nvd_entry)
    data['_id'] = data["id"]
    data.pop('id')
    data.pop('rejected')
    data['cwe'] = list(data['cwe'])
    data['cvssv2'] = str(data['cvssv2'])
    data['cvssv3'] = str(data['cvssv3'])
    return data

mongo_collection_all = mongo_db["nvd_all"]
#mongo_collection_all.drop()
for year in years:
    entries_not_rejected = [feed.get_cve_entry(entry) for entry in feed[year] if feed.get_cve_entry(entry).rejected == False]
    list_to_insert = [nvd_entry_to_dict(e) for e in entries_not_rejected]
    mongo_collection_name = f"nvd_{year}"
    mongo_collection = mongo_db[mongo_collection_name]
    mongo_collection.drop()
    result = mongo_collection.insert_many(list_to_insert)
    print(f'Acknowledged: {result.acknowledged}')
    ids: list = result.inserted_ids
    ids.sort()
    print(f'Inserted {len(result.inserted_ids)} documents - from {ids[0]} to {ids[-1]}')
#    mongo_collection_all.insert_many(list_to_insert)

Acknowledged: True
Inserted 6096 documents - from CVE-2022-0001 to CVE-2022-31489


In [6]:
c = mongo_db['nvd_2022'].count_documents({})
c

6096